In [3]:
!pip install monai
!pip install torch
!pip install torch-vision

In [5]:
!pip install pyradiomics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 35.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Discarding https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz (from https://pypi.org/simple/pyradiomics/): Requested pyradiomics from https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz has inconsistent version: expected '3.1.0', but metadata has '3.0.1a1'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.9/112.9 kB 12.5 MB/s eta 0:00:0

In [6]:
import SimpleITK as sitk
from radiomics import featureextractor
import pandas as pd

In [8]:
import os
import pandas as pd
import SimpleITK as sitk
from radiomics import featureextractor
from glob import glob

# Directory containing the image and mask files
data_dir = '/content/'

# Function to calculate radiomics features for a given image and mask
def calculate_radiomics_features(image_path, mask_path):
    try:
        # Load the image and mask using SimpleITK
        image = sitk.ReadImage(image_path)
        mask = sitk.ReadImage(mask_path)

        # Initialize the feature extractor
        extractor = featureextractor.RadiomicsFeatureExtractor()

        # Define the feature classes you want to enable
        extractor.disableAllFeatures()
        extractor.enableFeatureClassByName('firstorder')
        extractor.enableFeatureClassByName('shape')

        # Calculate radiomics features
        features = extractor.execute(image, mask)

        # Convert the features to a pandas DataFrame, excluding metadata
        features = {k: v for k, v in features.items() if not k.startswith('diagnostics_')}
        return features
    except ValueError as e:
        # Handle the geometry discrepancy error by returning None
        return None

# List of image and label file paths for training and validation
images = sorted(glob(os.path.join(data_dir, 'images', '*.nii.gz')))
labels = sorted(glob(os.path.join(data_dir, 'masks', '*.nii.gz')))


# List of dictionaries containing image and label file paths
train_files = [{"image": image_name, "label": label_name} for image_name, label_name in zip(images, labels)]

In [9]:
# Initialize an empty DataFrame to store the results
# Loop through the files for training and validation
all_results = pd.DataFrame()
for file_info in train_files:
    image_path = file_info["image"]
    label_path = file_info["label"]

    features = calculate_radiomics_features(image_path, label_path)

    if features is not None:
        # Create a DataFrame to store the results for this image
        results = pd.DataFrame({'Feature Name': features.keys(), 'Feature Value': features.values()})

        # Transpose the results
        results = results.T
        results.columns = results.iloc[0]
        results = results[1:]

        # Append the transposed results for this image to the overall results DataFrame
        all_results = all_results.append(results)

# Reset the index for the combined results DataFrame
all_results = all_results.reset_index(drop=True)

# Print or use the results DataFrame for further analysis
all_results

INFO:radiomics.featureextractor:No valid config parameter, using defaults: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}
INFO:radiomics.featureextractor:Enabled image types: {'Original': {}}
INFO:radiomics.featureextractor:Enabled features: {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firsto

Feature Name,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,...,original_firstorder_Mean,original_firstorder_Median,original_firstorder_Minimum,original_firstorder_Range,original_firstorder_RobustMeanAbsoluteDeviation,original_firstorder_RootMeanSquared,original_firstorder_Skewness,original_firstorder_TotalEnergy,original_firstorder_Uniformity,original_firstorder_Variance
0,0.753797,0.612982,57.331785,93.52925,111.54155971252712,94.06136228323378,94.05471284028752,112.5691741101709,216846.47848148597,70.502113,...,230.10372952474123,227.0,32.0,754.0,26.4738007046247,235.7716079202405,0.7861110099763104,12059227447.583439,0.145669087859489,2640.5247601003634
1,0.561102,0.521901,34.954346,66.974991,77.69203194371,76.92858597129738,45.79812391464717,79.17533509983365,52283.719892982794,37.579785,...,170.9482341865554,167.0,44.0,345.0,22.27923798238273,175.7168764755878,0.5821070872737725,1616090288.069726,0.17751749285528118,1653.1219068355952
2,0.633809,0.433765,13.40079,30.894093,28.561231117668893,29.70857174787974,27.35027435171559,33.9476024730172,4377.9408246633875,19.580949,...,910.9250535331906,890.0,300.0,1356.0,92.84036737955768,932.4066348768184,0.37195569991281324,3831764463.86354,0.04154779767461368,39597.679607866514
3,0.539027,0.523506,22.946799,43.832957,32.877628387499456,49.1087615337273,42.894689034565985,53.06209359361576,12724.042498833625,23.62713,...,526.5837979816598,517.0,172.0,801.0,65.41332252266774,538.3884281637788,0.32563903364992636,3703191872.6041174,0.06270804096140124,12571.603283874952
4,0.922744,0.882486,50.336813,57.039797,71.07819444091679,67.02054006316085,61.47112413579487,71.32287480057404,73935.62666281576,52.633112,...,87.21940019763741,88.0,17.0,208.0,7.277692282103722,88.42962776759342,0.14301237988691082,578531949.3256973,0.49735913343968813,212.57529627949447
5,0.750518,0.71447,55.62207,77.850845,88.39241823124986,89.3100411291216,67.46877090272075,92.01275838085061,174205.79072687615,58.428469,...,82.00449016353961,82.0,8.0,164.0,6.485014759033536,82.88355637218177,-0.04759696118463045,1197099186.3458116,0.5348697050655262,144.94750991856813
6,0.916182,0.488307,34.70989,71.082136,74.74327182647738,77.31731931433514,74.16444079222947,81.34778877684153,93721.19825306769,65.124173,...,117.30691342280645,118.0,2.0,345.0,8.207897313813293,118.7245360598296,0.06284050151359123,1321727616.459612,0.39167070423646533,334.60352583597137
7,0.845532,0.32376,21.834619,67.440816,66.17447278695485,73.02138993293109,69.35565031680488,74.91806299045236,47005.94857333613,57.023376,...,75.7104462068413,74.0,16.0,145.0,9.047938414438857,77.52436237796965,0.6841469757340176,282944348.4983995,0.3974538295715457,277.9550972717454
8,0.849776,0.799529,80.163845,100.263855,114.54983303461198,122.47693977410702,106.73843311310134,126.8881138274837,460178.52853538276,85.20179,...,85.56224579603273,85.0,0.0,218.0,7.026641406571424,86.68311473563963,0.3958123421912976,3458507192.4156137,0.5066764659038043,193.06447461134536
9,0.572733,0.382564,25.001439,65.352275,67.79941515323591,66.20313898687789,43.82491815116872,73.05018432030525,36163.081195657025,37.429432,...,77.39093362972372,78.0,11.0,169.0,7.892459959892475,78.52095431635654,-0.1431569672499841,223239566.74619108,0.46322561735083834,176.18365867104748


In [10]:
all_results

Feature Name,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,...,original_firstorder_Mean,original_firstorder_Median,original_firstorder_Minimum,original_firstorder_Range,original_firstorder_RobustMeanAbsoluteDeviation,original_firstorder_RootMeanSquared,original_firstorder_Skewness,original_firstorder_TotalEnergy,original_firstorder_Uniformity,original_firstorder_Variance
0,0.753797,0.612982,57.331785,93.52925,111.54155971252712,94.06136228323378,94.05471284028752,112.5691741101709,216846.47848148597,70.502113,...,230.10372952474123,227.0,32.0,754.0,26.4738007046247,235.7716079202405,0.7861110099763104,12059227447.583439,0.145669087859489,2640.5247601003634
1,0.561102,0.521901,34.954346,66.974991,77.69203194371,76.92858597129738,45.79812391464717,79.17533509983365,52283.719892982794,37.579785,...,170.9482341865554,167.0,44.0,345.0,22.27923798238273,175.7168764755878,0.5821070872737725,1616090288.069726,0.17751749285528118,1653.1219068355952
2,0.633809,0.433765,13.40079,30.894093,28.561231117668893,29.70857174787974,27.35027435171559,33.9476024730172,4377.9408246633875,19.580949,...,910.9250535331906,890.0,300.0,1356.0,92.84036737955768,932.4066348768184,0.37195569991281324,3831764463.86354,0.04154779767461368,39597.679607866514
3,0.539027,0.523506,22.946799,43.832957,32.877628387499456,49.1087615337273,42.894689034565985,53.06209359361576,12724.042498833625,23.62713,...,526.5837979816598,517.0,172.0,801.0,65.41332252266774,538.3884281637788,0.32563903364992636,3703191872.6041174,0.06270804096140124,12571.603283874952
4,0.922744,0.882486,50.336813,57.039797,71.07819444091679,67.02054006316085,61.47112413579487,71.32287480057404,73935.62666281576,52.633112,...,87.21940019763741,88.0,17.0,208.0,7.277692282103722,88.42962776759342,0.14301237988691082,578531949.3256973,0.49735913343968813,212.57529627949447
5,0.750518,0.71447,55.62207,77.850845,88.39241823124986,89.3100411291216,67.46877090272075,92.01275838085061,174205.79072687615,58.428469,...,82.00449016353961,82.0,8.0,164.0,6.485014759033536,82.88355637218177,-0.04759696118463045,1197099186.3458116,0.5348697050655262,144.94750991856813
6,0.916182,0.488307,34.70989,71.082136,74.74327182647738,77.31731931433514,74.16444079222947,81.34778877684153,93721.19825306769,65.124173,...,117.30691342280645,118.0,2.0,345.0,8.207897313813293,118.7245360598296,0.06284050151359123,1321727616.459612,0.39167070423646533,334.60352583597137
7,0.845532,0.32376,21.834619,67.440816,66.17447278695485,73.02138993293109,69.35565031680488,74.91806299045236,47005.94857333613,57.023376,...,75.7104462068413,74.0,16.0,145.0,9.047938414438857,77.52436237796965,0.6841469757340176,282944348.4983995,0.3974538295715457,277.9550972717454
8,0.849776,0.799529,80.163845,100.263855,114.54983303461198,122.47693977410702,106.73843311310134,126.8881138274837,460178.52853538276,85.20179,...,85.56224579603273,85.0,0.0,218.0,7.026641406571424,86.68311473563963,0.3958123421912976,3458507192.4156137,0.5066764659038043,193.06447461134536
9,0.572733,0.382564,25.001439,65.352275,67.79941515323591,66.20313898687789,43.82491815116872,73.05018432030525,36163.081195657025,37.429432,...,77.39093362972372,78.0,11.0,169.0,7.892459959892475,78.52095431635654,-0.1431569672499841,223239566.74619108,0.46322561735083834,176.18365867104748


In [12]:
s = all_results.head(54)
s

Feature Name,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,...,original_firstorder_Mean,original_firstorder_Median,original_firstorder_Minimum,original_firstorder_Range,original_firstorder_RobustMeanAbsoluteDeviation,original_firstorder_RootMeanSquared,original_firstorder_Skewness,original_firstorder_TotalEnergy,original_firstorder_Uniformity,original_firstorder_Variance
0,0.753797,0.612982,57.331785,93.52925,111.54155971252712,94.06136228323378,94.05471284028752,112.5691741101709,216846.47848148597,70.502113,...,230.10372952474123,227.0,32.0,754.0,26.4738007046247,235.7716079202405,0.7861110099763104,12059227447.583439,0.145669087859489,2640.5247601003634
1,0.561102,0.521901,34.954346,66.974991,77.69203194371,76.92858597129738,45.79812391464717,79.17533509983365,52283.719892982794,37.579785,...,170.9482341865554,167.0,44.0,345.0,22.27923798238273,175.7168764755878,0.5821070872737725,1616090288.069726,0.17751749285528118,1653.1219068355952
2,0.633809,0.433765,13.40079,30.894093,28.561231117668893,29.70857174787974,27.35027435171559,33.9476024730172,4377.9408246633875,19.580949,...,910.9250535331906,890.0,300.0,1356.0,92.84036737955768,932.4066348768184,0.37195569991281324,3831764463.86354,0.04154779767461368,39597.679607866514
3,0.539027,0.523506,22.946799,43.832957,32.877628387499456,49.1087615337273,42.894689034565985,53.06209359361576,12724.042498833625,23.62713,...,526.5837979816598,517.0,172.0,801.0,65.41332252266774,538.3884281637788,0.32563903364992636,3703191872.6041174,0.06270804096140124,12571.603283874952
4,0.922744,0.882486,50.336813,57.039797,71.07819444091679,67.02054006316085,61.47112413579487,71.32287480057404,73935.62666281576,52.633112,...,87.21940019763741,88.0,17.0,208.0,7.277692282103722,88.42962776759342,0.14301237988691082,578531949.3256973,0.49735913343968813,212.57529627949447
5,0.750518,0.71447,55.62207,77.850845,88.39241823124986,89.3100411291216,67.46877090272075,92.01275838085061,174205.79072687615,58.428469,...,82.00449016353961,82.0,8.0,164.0,6.485014759033536,82.88355637218177,-0.04759696118463045,1197099186.3458116,0.5348697050655262,144.94750991856813
6,0.916182,0.488307,34.70989,71.082136,74.74327182647738,77.31731931433514,74.16444079222947,81.34778877684153,93721.19825306769,65.124173,...,117.30691342280645,118.0,2.0,345.0,8.207897313813293,118.7245360598296,0.06284050151359123,1321727616.459612,0.39167070423646533,334.60352583597137
7,0.845532,0.32376,21.834619,67.440816,66.17447278695485,73.02138993293109,69.35565031680488,74.91806299045236,47005.94857333613,57.023376,...,75.7104462068413,74.0,16.0,145.0,9.047938414438857,77.52436237796965,0.6841469757340176,282944348.4983995,0.3974538295715457,277.9550972717454
8,0.849776,0.799529,80.163845,100.263855,114.54983303461198,122.47693977410702,106.73843311310134,126.8881138274837,460178.52853538276,85.20179,...,85.56224579603273,85.0,0.0,218.0,7.026641406571424,86.68311473563963,0.3958123421912976,3458507192.4156137,0.5066764659038043,193.06447461134536
9,0.572733,0.382564,25.001439,65.352275,67.79941515323591,66.20313898687789,43.82491815116872,73.05018432030525,36163.081195657025,37.429432,...,77.39093362972372,78.0,11.0,169.0,7.892459959892475,78.52095431635654,-0.1431569672499841,223239566.74619108,0.46322561735083834,176.18365867104748


In [13]:
s.columns

Index(['original_shape_Elongation', 'original_shape_Flatness',
       'original_shape_LeastAxisLength', 'original_shape_MajorAxisLength',
       'original_shape_Maximum2DDiameterColumn',
       'original_shape_Maximum2DDiameterRow',
       'original_shape_Maximum2DDiameterSlice',
       'original_shape_Maximum3DDiameter', 'original_shape_MeshVolume',
       'original_shape_MinorAxisLength', 'original_shape_Sphericity',
       'original_shape_SurfaceArea', 'original_shape_SurfaceVolumeRatio',
       'original_shape_VoxelVolume', 'original_firstorder_10Percentile',
       'original_firstorder_90Percentile', 'original_firstorder_Energy',
       'original_firstorder_Entropy', 'original_firstorder_InterquartileRange',
       'original_firstorder_Kurtosis', 'original_firstorder_Maximum',
       'original_firstorder_MeanAbsoluteDeviation', 'original_firstorder_Mean',
       'original_firstorder_Median', 'original_firstorder_Minimum',
       'original_firstorder_Range',
       'original_firsto

In [14]:
# Define a mapping of current column names to new names
column_mapping = {
    col: col.replace('original_shape_', '').replace('original_firstorder_', '') for col in s.columns
}

In [15]:
# Rename the columns
s.rename(columns=column_mapping, inplace=True)
s

<ipython-input-15-abad798e0c45>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s.rename(columns=column_mapping, inplace=True)


Feature Name,Elongation,Flatness,LeastAxisLength,MajorAxisLength,Maximum2DDiameterColumn,Maximum2DDiameterRow,Maximum2DDiameterSlice,Maximum3DDiameter,MeshVolume,MinorAxisLength,...,Mean,Median,Minimum,Range,RobustMeanAbsoluteDeviation,RootMeanSquared,Skewness,TotalEnergy,Uniformity,Variance
0,0.753797,0.612982,57.331785,93.52925,111.54155971252712,94.06136228323378,94.05471284028752,112.5691741101709,216846.47848148597,70.502113,...,230.10372952474123,227.0,32.0,754.0,26.4738007046247,235.7716079202405,0.7861110099763104,12059227447.583439,0.145669087859489,2640.5247601003634
1,0.561102,0.521901,34.954346,66.974991,77.69203194371,76.92858597129738,45.79812391464717,79.17533509983365,52283.719892982794,37.579785,...,170.9482341865554,167.0,44.0,345.0,22.27923798238273,175.7168764755878,0.5821070872737725,1616090288.069726,0.17751749285528118,1653.1219068355952
2,0.633809,0.433765,13.40079,30.894093,28.561231117668893,29.70857174787974,27.35027435171559,33.9476024730172,4377.9408246633875,19.580949,...,910.9250535331906,890.0,300.0,1356.0,92.84036737955768,932.4066348768184,0.37195569991281324,3831764463.86354,0.04154779767461368,39597.679607866514
3,0.539027,0.523506,22.946799,43.832957,32.877628387499456,49.1087615337273,42.894689034565985,53.06209359361576,12724.042498833625,23.62713,...,526.5837979816598,517.0,172.0,801.0,65.41332252266774,538.3884281637788,0.32563903364992636,3703191872.6041174,0.06270804096140124,12571.603283874952
4,0.922744,0.882486,50.336813,57.039797,71.07819444091679,67.02054006316085,61.47112413579487,71.32287480057404,73935.62666281576,52.633112,...,87.21940019763741,88.0,17.0,208.0,7.277692282103722,88.42962776759342,0.14301237988691082,578531949.3256973,0.49735913343968813,212.57529627949447
5,0.750518,0.71447,55.62207,77.850845,88.39241823124986,89.3100411291216,67.46877090272075,92.01275838085061,174205.79072687615,58.428469,...,82.00449016353961,82.0,8.0,164.0,6.485014759033536,82.88355637218177,-0.04759696118463045,1197099186.3458116,0.5348697050655262,144.94750991856813
6,0.916182,0.488307,34.70989,71.082136,74.74327182647738,77.31731931433514,74.16444079222947,81.34778877684153,93721.19825306769,65.124173,...,117.30691342280645,118.0,2.0,345.0,8.207897313813293,118.7245360598296,0.06284050151359123,1321727616.459612,0.39167070423646533,334.60352583597137
7,0.845532,0.32376,21.834619,67.440816,66.17447278695485,73.02138993293109,69.35565031680488,74.91806299045236,47005.94857333613,57.023376,...,75.7104462068413,74.0,16.0,145.0,9.047938414438857,77.52436237796965,0.6841469757340176,282944348.4983995,0.3974538295715457,277.9550972717454
8,0.849776,0.799529,80.163845,100.263855,114.54983303461198,122.47693977410702,106.73843311310134,126.8881138274837,460178.52853538276,85.20179,...,85.56224579603273,85.0,0.0,218.0,7.026641406571424,86.68311473563963,0.3958123421912976,3458507192.4156137,0.5066764659038043,193.06447461134536
9,0.572733,0.382564,25.001439,65.352275,67.79941515323591,66.20313898687789,43.82491815116872,73.05018432030525,36163.081195657025,37.429432,...,77.39093362972372,78.0,11.0,169.0,7.892459959892475,78.52095431635654,-0.1431569672499841,223239566.74619108,0.46322561735083834,176.18365867104748


In [25]:
s.columns

Index(['Elongation', 'Flatness', 'LeastAxisLength', 'MajorAxisLength',
       'Maximum2DDiameterColumn', 'Maximum2DDiameterRow',
       'Maximum2DDiameterSlice', 'Maximum3DDiameter', 'MeshVolume',
       'MinorAxisLength', 'Sphericity', 'SurfaceArea', 'SurfaceVolumeRatio',
       'VoxelVolume', '10Percentile', '90Percentile', 'Energy', 'Entropy',
       'InterquartileRange', 'Kurtosis', 'Maximum', 'MeanAbsoluteDeviation',
       'Mean', 'Median', 'Minimum', 'Range', 'RobustMeanAbsoluteDeviation',
       'RootMeanSquared', 'Skewness', 'TotalEnergy', 'Uniformity', 'Variance'],
      dtype='object', name='Feature Name')

In [26]:
f = s.copy()

In [27]:
# To delete multiple columns 'B' and 'C'
f = s.drop(columns=['LeastAxisLength', 'Maximum2DDiameterColumn', 'Maximum2DDiameterRow',
       'Maximum2DDiameterSlice', 'MinorAxisLength', '10Percentile', '90Percentile', 'Maximum',
       'Minimum', 'Range', 'RobustMeanAbsoluteDeviation',
       'TotalEnergy'], axis=1)
f.head()

Feature Name,Elongation,Flatness,MajorAxisLength,Maximum3DDiameter,MeshVolume,Sphericity,SurfaceArea,SurfaceVolumeRatio,VoxelVolume,Energy,Entropy,InterquartileRange,Kurtosis,MeanAbsoluteDeviation,Mean,Median,RootMeanSquared,Skewness,Uniformity,Variance
0,0.753797,0.612982,93.52925,112.5691741101709,216846.47848148597,0.49025131069467215,35604.12170237441,0.16419045377955832,216938.421495,18865151541.0,3.051192295081715,64.0,6.349840061144612,39.217317184064974,230.10372952474123,227.0,235.7716079202405,0.7861110099763104,0.145669087859489,2640.5247601003634
1,0.561102,0.521901,66.974991,79.17533509983365,52283.719892982794,0.502944743567906,13444.402432897074,0.25714318836562927,52340.596888,2427010171.0,2.7342593126194,53.0,3.58940338832299,32.051724726068905,170.9482341865554,167.0,175.7168764755878,0.5821070872737725,0.17751749285528118,1653.1219068355952
2,0.633809,0.433765,30.894093,33.9476024730172,4377.9408246633875,0.5805177905730949,2229.368440369942,0.5092276322719265,4407.457112,6902024752.0,4.955359861861691,212.0,3.6887704279173685,148.8941012479255,910.9250535331906,890.0,932.4066348768184,0.37195569991281324,0.04154779767461368,39597.679607866514
3,0.539027,0.523506,43.832957,53.06209359361576,12724.042498833625,0.5709867353825179,4616.09614593089,0.3627853448582896,12775.702232,6290297423.0,4.1919228514575355,156.0,2.8349281279431713,90.95676107288894,526.5837979816598,517.0,538.3884281637788,0.32563903364992636,0.06270804096140124,12571.603283874952
4,0.922744,0.882486,57.039797,71.32287480057404,73935.62666281576,0.4272426672584733,19939.396627192134,0.26968590823103417,73982.968662,1725094593.0,1.364922062591215,17.0,5.119010912775989,10.90915161767479,87.21940019763741,88.0,88.42962776759342,0.14301237988691082,0.49735913343968813,212.57529627949447


In [28]:
f.rename(columns={'Feature Name': 'PatientID', 'MajorAxisLength': 'Length', 'Maximum3DDiameter': 'Diameter', 'InterquartileRange': 'IQR', 'RootMeanSquared': 'RMS'}, inplace=True)
f.head()

Feature Name,Elongation,Flatness,Length,Diameter,MeshVolume,Sphericity,SurfaceArea,SurfaceVolumeRatio,VoxelVolume,Energy,Entropy,IQR,Kurtosis,MeanAbsoluteDeviation,Mean,Median,RMS,Skewness,Uniformity,Variance
0,0.753797,0.612982,93.52925,112.5691741101709,216846.47848148597,0.49025131069467215,35604.12170237441,0.16419045377955832,216938.421495,18865151541.0,3.051192295081715,64.0,6.349840061144612,39.217317184064974,230.10372952474123,227.0,235.7716079202405,0.7861110099763104,0.145669087859489,2640.5247601003634
1,0.561102,0.521901,66.974991,79.17533509983365,52283.719892982794,0.502944743567906,13444.402432897074,0.25714318836562927,52340.596888,2427010171.0,2.7342593126194,53.0,3.58940338832299,32.051724726068905,170.9482341865554,167.0,175.7168764755878,0.5821070872737725,0.17751749285528118,1653.1219068355952
2,0.633809,0.433765,30.894093,33.9476024730172,4377.9408246633875,0.5805177905730949,2229.368440369942,0.5092276322719265,4407.457112,6902024752.0,4.955359861861691,212.0,3.6887704279173685,148.8941012479255,910.9250535331906,890.0,932.4066348768184,0.37195569991281324,0.04154779767461368,39597.679607866514
3,0.539027,0.523506,43.832957,53.06209359361576,12724.042498833625,0.5709867353825179,4616.09614593089,0.3627853448582896,12775.702232,6290297423.0,4.1919228514575355,156.0,2.8349281279431713,90.95676107288894,526.5837979816598,517.0,538.3884281637788,0.32563903364992636,0.06270804096140124,12571.603283874952
4,0.922744,0.882486,57.039797,71.32287480057404,73935.62666281576,0.4272426672584733,19939.396627192134,0.26968590823103417,73982.968662,1725094593.0,1.364922062591215,17.0,5.119010912775989,10.90915161767479,87.21940019763741,88.0,88.42962776759342,0.14301237988691082,0.49735913343968813,212.57529627949447


In [29]:
# Change the index name to 'NewIndexName'
y = f.copy()
y.index = y.index + 1
y.head()

Feature Name,Elongation,Flatness,Length,Diameter,MeshVolume,Sphericity,SurfaceArea,SurfaceVolumeRatio,VoxelVolume,Energy,Entropy,IQR,Kurtosis,MeanAbsoluteDeviation,Mean,Median,RMS,Skewness,Uniformity,Variance
1,0.753797,0.612982,93.52925,112.5691741101709,216846.47848148597,0.49025131069467215,35604.12170237441,0.16419045377955832,216938.421495,18865151541.0,3.051192295081715,64.0,6.349840061144612,39.217317184064974,230.10372952474123,227.0,235.7716079202405,0.7861110099763104,0.145669087859489,2640.5247601003634
2,0.561102,0.521901,66.974991,79.17533509983365,52283.719892982794,0.502944743567906,13444.402432897074,0.25714318836562927,52340.596888,2427010171.0,2.7342593126194,53.0,3.58940338832299,32.051724726068905,170.9482341865554,167.0,175.7168764755878,0.5821070872737725,0.17751749285528118,1653.1219068355952
3,0.633809,0.433765,30.894093,33.9476024730172,4377.9408246633875,0.5805177905730949,2229.368440369942,0.5092276322719265,4407.457112,6902024752.0,4.955359861861691,212.0,3.6887704279173685,148.8941012479255,910.9250535331906,890.0,932.4066348768184,0.37195569991281324,0.04154779767461368,39597.679607866514
4,0.539027,0.523506,43.832957,53.06209359361576,12724.042498833625,0.5709867353825179,4616.09614593089,0.3627853448582896,12775.702232,6290297423.0,4.1919228514575355,156.0,2.8349281279431713,90.95676107288894,526.5837979816598,517.0,538.3884281637788,0.32563903364992636,0.06270804096140124,12571.603283874952
5,0.922744,0.882486,57.039797,71.32287480057404,73935.62666281576,0.4272426672584733,19939.396627192134,0.26968590823103417,73982.968662,1725094593.0,1.364922062591215,17.0,5.119010912775989,10.90915161767479,87.21940019763741,88.0,88.42962776759342,0.14301237988691082,0.49735913343968813,212.57529627949447


In [30]:
y

Feature Name,Elongation,Flatness,Length,Diameter,MeshVolume,Sphericity,SurfaceArea,SurfaceVolumeRatio,VoxelVolume,Energy,Entropy,IQR,Kurtosis,MeanAbsoluteDeviation,Mean,Median,RMS,Skewness,Uniformity,Variance
1,0.753797,0.612982,93.52925,112.5691741101709,216846.47848148597,0.49025131069467215,35604.12170237441,0.16419045377955832,216938.421495,18865151541.0,3.051192295081715,64.0,6.349840061144612,39.217317184064974,230.10372952474123,227.0,235.7716079202405,0.7861110099763104,0.145669087859489,2640.5247601003634
2,0.561102,0.521901,66.974991,79.17533509983365,52283.719892982794,0.502944743567906,13444.402432897074,0.25714318836562927,52340.596888,2427010171.0,2.7342593126194,53.0,3.58940338832299,32.051724726068905,170.9482341865554,167.0,175.7168764755878,0.5821070872737725,0.17751749285528118,1653.1219068355952
3,0.633809,0.433765,30.894093,33.9476024730172,4377.9408246633875,0.5805177905730949,2229.368440369942,0.5092276322719265,4407.457112,6902024752.0,4.955359861861691,212.0,3.6887704279173685,148.8941012479255,910.9250535331906,890.0,932.4066348768184,0.37195569991281324,0.04154779767461368,39597.679607866514
4,0.539027,0.523506,43.832957,53.06209359361576,12724.042498833625,0.5709867353825179,4616.09614593089,0.3627853448582896,12775.702232,6290297423.0,4.1919228514575355,156.0,2.8349281279431713,90.95676107288894,526.5837979816598,517.0,538.3884281637788,0.32563903364992636,0.06270804096140124,12571.603283874952
5,0.922744,0.882486,57.039797,71.32287480057404,73935.62666281576,0.4272426672584733,19939.396627192134,0.26968590823103417,73982.968662,1725094593.0,1.364922062591215,17.0,5.119010912775989,10.90915161767479,87.21940019763741,88.0,88.42962776759342,0.14301237988691082,0.49735913343968813,212.57529627949447
6,0.750518,0.71447,77.850845,92.01275838085061,174205.79072687615,0.696348896523523,21662.020253271356,0.12434730305397006,174258.262946,4164498566.0,1.184866581617473,15.0,3.9718749232913173,9.2980687088977,82.00449016353961,82.0,82.88355637218177,-0.04759696118463045,0.5348697050655262,144.94750991856813
7,0.916182,0.488307,71.082136,81.34778877684153,93721.19825306769,0.4836873702439531,20628.988417290177,0.22011016506198955,93769.37083,4598041717.0,1.6689275376394612,19.0,6.180616308236658,13.185635690273738,117.30691342280645,118.0,118.7245360598296,0.06284050151359123,0.39167070423646533,334.60352583597137
8,0.845532,0.32376,67.440816,74.91806299045236,47005.94857333613,0.4808918886378446,13098.007058851896,0.2786457343460923,47078.716901,1050324297.0,1.5505230630419415,21.0,3.7137935989168427,13.032509123544305,75.7104462068413,74.0,77.52436237796965,0.6841469757340176,0.3974538295715457,277.9550972717454
9,0.849776,0.799529,100.263855,126.8881138274837,460178.52853538276,0.6087492342786289,47351.08453518888,0.10289720532136494,460277.416546,12985547132.0,1.320592153592527,16.0,5.048065530821859,10.48060027427841,85.56224579603273,85.0,86.68311473563963,0.3958123421912976,0.5066764659038043,193.06447461134536
10,0.572733,0.382564,65.352275,73.05018432030525,36163.081195657025,0.5525086436720142,9571.690900410302,0.26468128776482147,36207.624488,776611452.0,1.2836718895231831,19.0,3.4940723625409986,10.69862479391049,77.39093362972372,78.0,78.52095431635654,-0.1431569672499841,0.46322561735083834,176.18365867104748


In [38]:
y.columns

Index(['Elongation', 'Flatness', 'Length', 'Diameter', 'MeshVolume',
       'Sphericity', 'SurfaceArea', 'SurfaceVolumeRatio', 'VoxelVolume',
       'Energy', 'Entropy', 'IQR', 'Kurtosis', 'MeanAbsoluteDeviation', 'Mean',
       'Median', 'RMS', 'Skewness', 'Uniformity', 'Variance'],
      dtype='object', name='Feature Name')

In [37]:
y.to_csv('statisticalanalysis.csv', index=True)